In [1]:
import numpy as np
import time
import matplotlib.pyplot as plt
import os
import errno
import utils

import CGDs
import importlib
importlib.reload(CGDs)


from pyDOE import lhs
from torch import from_numpy

import torch
import torch.cuda
import torch.nn as nn
import torch.optim as optim
import torch.autograd as tgrad

# Manage device

In [2]:
os.environ['KMP_DUPLICATE_LIB_OK']='True'
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(torch.cuda.is_available())
# torch.set_default_tensor_type(torch.DoubleTensor)
print(device)

if device == 'cuda': 
    print(torch.cuda.get_device_name())

True
cuda


In [3]:
# utils.printMemory()

# Data Sampling

In [4]:
samples = {"pde": 50000, "bc":5000, "fc":5000}

K = 40.0
r = 0.05
sigma = 0.25
T = 1.0
S_range = [0.0, 130.0]
t_range = [0.0, T]
gs = lambda x: np.fmax(x-K, 0.0)

# Build Neural Network

In [5]:
import networks
# Create the model
PINNGACGD = networks.FeedforwardNeuralNetwork(2, 50, 1, 3)
PINNGACGD.to(device)
print(PINNGACGD)

FeedforwardNeuralNetwork(
  (layers): ModuleList(
    (0): Linear(in_features=2, out_features=50, bias=True)
    (1-2): 2 x Linear(in_features=50, out_features=50, bias=True)
  )
  (output): Linear(in_features=50, out_features=1, bias=True)
  (relu): ReLU()
)


In [6]:
D_GACGD = networks.Discriminator(2, 25, 1)
D_GACGD.to(device)
D_GACGD.load_state_dict(D_GACGD.state_dict()) # copy weights and stuff
print(D_GACGD)

Discriminator(
  (map): Sequential(
    (0): Linear(in_features=2, out_features=50, bias=True)
    (1): ReLU()
    (2): Linear(in_features=50, out_features=50, bias=True)
    (3): ReLU()
    (4): Linear(in_features=50, out_features=50, bias=True)
    (5): ReLU()
    (6): Linear(in_features=50, out_features=50, bias=True)
    (7): ReLU()
    (8): Linear(in_features=50, out_features=1, bias=True)
  )
)


# Network Trainig

### Hyperparameters

In [7]:
max_iter = 30000

# Define loss function and optimizer
tol = 1e-7
atol = 1e-20
g_iter = 1000
lr = 0.004
track_cond = lambda x, y:  True

optimizer = CGDs.GACGD(x_params=D_GACGD.parameters(), y_params = PINNGACGD.parameters(), max_iter = g_iter,
            lr_x=lr, lr_y=lr, tol=tol, atol = atol, eps=1e-8, beta=0.99, track_cond = track_cond)
lossFunction = nn.MSELoss()
lossfunction2 = nn.L1Loss()

### Training

In [8]:
start_time=time.time()
loss_hist = []

for epoch in range(max_iter):
    
    optimizer.zero_grad() # zeroes the gradient buffers of all parameters
    
    # sampling
    bc_st_train, bc_v_train, n_st_train, n_v_train = \
    utils.trainingData(K, 
                       r, 
                       sigma, 
                       T, 
                       S_range[-1], 
                       S_range, 
                       t_range, 
                       gs, 
                       samples['bc'], 
                       samples['fc'], 
                       samples['pde'], 
                       RNG_key=123)
    
    # save training data points to tensor and send to device
    n_st_train = torch.from_numpy(n_st_train).float().requires_grad_().to(device)
    n_v_train = torch.from_numpy(n_v_train).float().to(device)
    
    bc_st_train = torch.from_numpy(bc_st_train).float().to(device)
    bc_v_train = torch.from_numpy(bc_v_train).float().to(device)
    
    
    # normal loss
    # print(n_st_train)
    # print(PINNBCGD.output.weight.dtype)
    v1_hat = PINNGACGD(n_st_train)
    
    grads = tgrad.grad(v1_hat, n_st_train, grad_outputs=torch.ones(v1_hat.shape).cuda(), 
                       retain_graph=True, create_graph=True, only_inputs=True)[0]
    dVdt, dVdS = grads[:, 0].view(-1, 1), grads[:, 1].view(-1, 1)
    grads2nd = tgrad.grad(dVdS, n_st_train, grad_outputs=torch.ones(dVdS.shape).cuda(), create_graph=True, only_inputs=True)[0]
    d2VdS2 = grads2nd[:, 1].view(-1, 1)
    S1 = n_st_train[:, 1].view(-1, 1)
    pde_loss = lossFunction(-dVdt, 0.5*((sigma*S1)**2)*d2VdS2 + r*S1*dVdS - r*v1_hat)
    
    loss1 = D_GACGD(n_st_train) * (dVdt + 0.5*((sigma*S1)**2)*d2VdS2 + r*S1*dVdS - r*v1_hat)
    
    
    # boundary condition loss
    bc_hat = PINNGACGD(bc_st_train)
    # print(bc_v_train)
    # print('111111111111111111111')
    # print(bc_hat)
    bc_loss = lossFunction(bc_v_train, bc_hat)
    
    loss2 = D_GACGD(bc_st_train) * (bc_hat - bc_v_train)
    
    
    # Backpropagation and Update
    combined_loss = (loss1.mean() + loss2.mean())
    pinn_loss = pde_loss.mean() + bc_loss.mean()
    # combined_loss.backward()
    optimizer.step(-combined_loss, combined_loss, 0)
    
    loss_hist.append(combined_loss.item())
    if epoch % 50 == 0:
        print(f'{epoch}/{max_iter} PDE Loss: {pde_loss.item():.5f}, BC Loss: {bc_loss.item():.5f}, \
                  mse loss: {pinn_loss.item():5f}')
        pass
        
end_time = time.time()
print('run time:', end_time - start_time)
print('finish')

c:\Users\hengr\anaconda3\lib\site-packages\CGDs\gmres_torch.py:124: UserWarning: torch.triangular_solve is deprecated in favor of torch.linalg.solve_triangularand will be removed in a future PyTorch release.
torch.linalg.solve_triangular has its arguments reversed and does not return a copy of one of the inputs.
X = torch.triangular_solve(B, A).solution
should be replaced with
X = torch.linalg.solve_triangular(A, B). (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\BatchLinearAlgebra.cpp:2197.)
  y, _ = torch.triangular_solve(beta[0:j + 1].unsqueeze(-1), H[0:j + 1, 0:j + 1])  # j x j


0/30000 PDE Loss: 0.00043, BC Loss: 3355.69971,                   mse loss: 3355.700195, nn loss: 31.829561
50/30000 PDE Loss: 29.58006, BC Loss: 54.00415,                   mse loss: 83.584213, nn loss: 0.989069
100/30000 PDE Loss: 7.43215, BC Loss: 6.53805,                   mse loss: 13.970201, nn loss: -31.367092
150/30000 PDE Loss: 11.27877, BC Loss: 15.98076,                   mse loss: 27.259525, nn loss: 22.534605
200/30000 PDE Loss: 6.26722, BC Loss: 5.74800,                   mse loss: 12.015221, nn loss: 10.084473
250/30000 PDE Loss: 2.82239, BC Loss: 1.46543,                   mse loss: 4.287813, nn loss: 28.946959
300/30000 PDE Loss: 0.38315, BC Loss: 0.19079,                   mse loss: 0.573934, nn loss: 20.032360
350/30000 PDE Loss: 0.07494, BC Loss: 0.04639,                   mse loss: 0.121323, nn loss: 7.415416
400/30000 PDE Loss: 0.06898, BC Loss: 0.02136,                   mse loss: 0.090336, nn loss: 1.762081
450/30000 PDE Loss: 0.08340, BC Loss: 0.01771,         

KeyboardInterrupt: 